# Week 10. Databases and SQL
## Today's objectives

1. Understand when and why you would want to use a database, rather than `pandas`
2. Learn how to set up a PostgreSQL and PostGIS database
3. Learn some basic SQL queries
4. Introduce how to integrate a database with Python and QGIS

Today's course will be mostly a taster. We won't do much on coding in SQL. Rather, I want to give you enough background so that you'll know when you should go further, and where to start.

One takeway: you should never need to pay money for ArcGIS. The open-source substitutes are fabulous! (But as long as most public agencies continue to think otherwise, ArcGIS is still an important skill for planners.)

You'll need to install the `pandasql` library. 

From the Terminal (Mac) or Anaconda Prompt (Windows):
   
(1) Switch to your environment (skip this if you don't have a separate environment set up for class)

Windows:
`activate your_environment_name` 

Mac:
`conda activate your_environment_name` 

(2) Install

`conda install pandasql -c conda-forge`

## Why use a database
In most cases, Python in general, and `pandas`/`geopandas` in particular, will do an excellent job for your data science workflow. But sometimes, they might not be the best option. Here are some situations when you might want to consider a database.
* Your dataset won't fit in memory
* Multiple users need access at the same time
* You need to validate and maintain the integrity of the data through access controls and transactions (a transaction means that if the computer crashes halfway through, your data integrity is unaffected)

Many planning agencies have databases already hosted on the agency's server, for example to store land-use permits or GIS files. Some of these might be spatial databases, e.g. with ESRI's ArcGIS suite. You might need to query these. 

You can also set up your own database, even on a laptop. External hard drives are pretty cheap. Of course, if you want someone else to access it, an always-on computer with a stable internet connection is needed. But this could even be an old computer that you keep in the corner of the office or lab.

Key takeaway: Data that needs to be preserved and maintained, whether permit issuance, rider surveys, or asset inventories such as bus stops, shouldn't be an Excel file on someone's computer. What happens if they retire or their computer gets stolen? Or someone inadvertently "corrects" the data and introduces errors? Or different users have different versions of the Excel file on their computers?

## How to set up a database
I highly recommend PostgreSQL (Postgres for short). It's free, open-source, and cross-platform (runs on Windows, Mac, and Linux.) It is well supported and updated, and has incredible spatial analysis capabilities through the [PostGIS](http://postgis.net) extension.

Most databases use the same language — SQL, which stands for Structured Query Language. So if you learn Postgres, you should be able to use pretty much any alternative with minimal hassle.

Installing Postgres is pretty simple. We won't need to do it for this class. But if you feel inclined to experiment, [the PostGIS site provides a list of downloadable versions](http://postgis.net/install/). For Mac, I recommend postgres.app to get started, as it works just like any Mac app, and the Homebrew version for more advanced users.

You don't need any sort of fancy computer to download and run Postgres. If you have large datasets or intensive computations, almost any cloud provider (e.g. AWS) lets you run Postgres.

## Database structure and access
Postgres organizes your data into:
* Databases. For example, I usually have one database per research project.
* Schemas. These are like directories on your computer, to organize your tables. For example, you might have schemas for `inputdata`, `working`, and `output`.
* Tables. Each Postgres table is like a `(geo)pandas` `DataFrame`.

How do you access the data? We will demonstrate a couple of these options in class:
* The graphical interface `pgAdmin 4`, which runs in a web browser
* The command line (`psql`)
* If you have spatial data, [QGIS](https://qgis.org/en/site/), an open-source alternative to ArcMap
* And of course...Python. `pandas` can read and write database tables with the `read_sql_query` and `to_sql` functions. For more flexibility, the `psycopg2` (soon-to-be `psycopg3`) library provides more power.

## Basic SQL
Even if you don't set up your own database, it can be useful to know how to query a database using SQL. The intuition is pretty similar to `pandas` — particularly with joins and `groupby` functions.

Here, we will walk through some practical examples. But rather than querying a database using SQL, we'll query a `pandas` dataframe using SQL, using . 

We'll use the same Ventura County collisions data from a couple of weeks ago. [Here's a refresher on the data structure.](https://tims.berkeley.edu/help/SWITRS.php)

In [12]:
import pandas as pd
from pandasql import sqldf
path = '../week3/'
collisionDf = pd.read_csv(path+'Collisions.csv')
partiesDf   = pd.read_csv(path+'Parties.csv')
victimsDf   = pd.read_csv(path+'Victims.csv')

# let's convert the columns to lower case to make them easier to read
# also, in Postgres, all column names are lower case
collisionDf.columns = collisionDf.columns.str.lower()
partiesDf.columns   = partiesDf.columns.str.lower()
victimsDf.columns   = victimsDf.columns.str.lower()

Index(['case_id', 'accident_year', 'proc_date', 'juris', 'collision_date',
       'collision_time', 'officer_id', 'reporting_district', 'day_of_week',
       'chp_shift', 'population', 'cnty_city_loc', 'special_cond', 'beat_type',
       'chp_beat_type', 'city_division_lapd', 'chp_beat_class', 'beat_number',
       'primary_rd', 'secondary_rd', 'distance', 'direction', 'intersection',
       'weather_1', 'weather_2', 'state_hwy_ind', 'caltrans_county',
       'caltrans_district', 'state_route', 'route_suffix', 'postmile_prefix',
       'postmile', 'location_type', 'ramp_intersection', 'side_of_hwy',
       'tow_away', 'collision_severity', 'number_killed', 'number_injured',
       'party_count', 'primary_coll_factor', 'pcf_code_of_viol',
       'pcf_viol_category', 'pcf_violation', 'pcf_viol_subsection',
       'hit_and_run', 'type_of_collision', 'mviw', 'ped_action',
       'road_surface', 'road_cond_1', 'road_cond_2', 'lighting',
       'control_device', 'chp_road_type', 'pedestrian_

In [6]:
collisionDf.head()

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,COUNT_MC_KILLED,COUNT_MC_INJURED,PRIMARY_RAMP,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y
0,8167180,2018,2020-05-26,5600,2018-01-29,1700,3386,8125,1,5,...,0,0,-,-,34.220058,-119.038971,VENTURA,CAMARILLO,-119.039001,34.219994
1,8414912,2018,2018-02-14,5600,2018-01-30,1840,2740,9080,2,5,...,0,0,-,-,NaN,NaN,VENTURA,THOUSAND OAKS,-118.883827,34.182751
2,8414916,2018,2018-02-14,5600,2018-01-22,1425,2740,9122,1,5,...,0,0,-,-,NaN,NaN,VENTURA,THOUSAND OAKS,-118.926300,34.182961
3,8504531,2018,2018-08-15,5600,2018-01-31,2358,5002,8153,3,5,...,0,0,-,-,34.205349,-119.040993,VENTURA,CAMARILLO,-119.040749,34.205418
4,8513493,2018,2018-02-15,5608,2018-01-01,1801,492,NaN,1,5,...,0,0,-,-,34.280899,-119.285400,VENTURA,VENTURA,NaN,NaN


### SELECT...FROM
The simplest SQL queries take this form:

`SELECT column1, column2...FROM tablename;`

Note the `;` at the end of each query.

For example, suppose we want to get two columns from `collisionDf`: `collision_date` and `officer_id`.

In [17]:
# exactly the same as collisionDf[['collision_date', 'officer_id']]
sqldf('SELECT collision_date, officer_id FROM collisionDf;')

,COLLISION_DATE,OFFICER_ID
0,2018-01-29,3386
1,2018-01-30,2740
2,2018-01-22,2740
3,2018-01-31,5002
4,2018-01-01,492
...,...,...
315,2018-01-23,14823
316,2018-01-31,15117
317,2018-01-17,15117
318,2018-01-24,17973


### WHERE
What if we only want to get a subset of rows? Use `WHERE`.

`SELECT column1, column2...FROM tablename WHERE condition;`

For example, suppose we want to get the two columns where a pedestrian is involved. From the documentation, this is any value except `A` in the `ped_action` column.

In [18]:
# exactly the same as collisionDf[collisionDf.ped_action!='A'][['collision_date', 'officer_id']]
sqldf('SELECT collision_date, officer_id FROM collisionDf WHERE ped_action!="A";')

,COLLISION_DATE,OFFICER_ID
0,2018-01-29,3386
1,2018-01-01,492
2,2018-01-03,462
3,2018-01-16,4550
4,2018-01-09,456
5,2018-01-12,497
6,2018-01-13,459
7,2018-01-04,2833
8,2018-01-19,438
9,2018-01-11,437


### Other useful queries
* Rather than a column name, `*` gives you all columns
* `DISTINCT` gives unique values
* `LIMIT` limits the number of rows returned (e.g. `LIMIT 10`)
* `COUNT(*)` gives the number of rows

In [19]:
sqldf('SELECT DISTINCT officer_id FROM collisionDf;')

,OFFICER_ID
0,3386
1,2740
2,5002
3,492
4,2994
...,...
171,19827
172,20238
173,16727
174,14535


In [21]:
sqldf('SELECT DISTINCT officer_id FROM collisionDf LIMIT 2;')

,OFFICER_ID
0,3386
1,2740


In [22]:
sqldf('SELECT * FROM collisionDf LIMIT 2;')

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,COUNT_MC_KILLED,COUNT_MC_INJURED,PRIMARY_RAMP,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y
0,8167180,2018,2020-05-26,5600,2018-01-29,1700,3386,8125,1,5,...,0,0,-,-,34.220058,-119.038971,VENTURA,CAMARILLO,-119.039001,34.219994
1,8414912,2018,2018-02-14,5600,2018-01-30,1840,2740,9080,2,5,...,0,0,-,-,NaN,NaN,VENTURA,THOUSAND OAKS,-118.883827,34.182751


### GROUP BY

`GROUP BY` works the same way, at least conceptually as in `pandas`

In [24]:
# same as collisionDf.groupby('juris').size()
sqldf('SELECT juris, COUNT(*) AS n FROM collisionDf GROUP BY juris;')

,JURIS,n
0,5600,75
1,5604,81
2,5605,3
3,5606,4
4,5608,48
5,5609,19
6,9765,63
7,9770,27


### JOINS
Finally, we can join tables together. Normally, to save disk space, we'd do this on the fly, rather than creating new joined tables.

The join concepts, such as left joins, inner joins, 1:1 joins, are the same as in `pandas`.

In [27]:
# note the ''' allows a string to go over multiple lines, for readability
sqldf('''SELECT *
         FROM collisionDf 
         LEFT JOIN partiesDf USING (case_id);''')

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,VEHICLE_YEAR,VEHICLE_MAKE,STWD_VEHICLE_TYPE,CHP_VEH_TYPE_TOWING,CHP_VEH_TYPE_TOWED,RACE,INATTENTION,SPECIAL_INFO_F,SPECIAL_INFO_G,ACCIDENT_YEAR
0,8167180,2018,2020-05-26,5600,2018-01-29,1700,3386,8125,1,5,...,NaN,-,N,60.0,NaN,H,None,-,-,2018
1,8167180,2018,2020-05-26,5600,2018-01-29,1700,3386,8125,1,5,...,2016.0,MAZDA,A,7.0,NaN,W,None,-,-,2018
2,8414912,2018,2018-02-14,5600,2018-01-30,1840,2740,9080,2,5,...,2017.0,HONDA,A,1.0,NaN,O,None,-,-,2018
3,8414912,2018,2018-02-14,5600,2018-01-30,1840,2740,9080,2,5,...,2001.0,FORD,D,22.0,NaN,W,None,-,-,2018
4,8414916,2018,2018-02-14,5600,2018-01-22,1425,2740,9122,1,5,...,2003.0,TOYOTA,D,22.0,NaN,W,None,-,-,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,90787677,2018,2018-08-09,9770,2018-01-08,1605,19276,None,1,2,...,2011.0,MERB,A,1.0,NaN,H,None,-,-,2018
628,90787677,2018,2018-08-09,9770,2018-01-08,1605,19276,None,1,2,...,2014.0,CHEV,A,1.0,NaN,O,None,-,-,2018
629,90787677,2018,2018-08-09,9770,2018-01-08,1605,19276,None,1,2,...,2017.0,DODG,D,22.0,NaN,W,None,-,-,2018
630,90787677,2018,2018-08-09,9770,2018-01-08,1605,19276,None,1,2,...,2006.0,TOYO,A,1.0,NaN,H,None,-,-,2018


### Spatial operations
PostGIS supports all the spatial operations that are in `geopandas`, and many more. For example:

`SELECT * FROM parcels, cities WHERE ST_Intersects(parcels.geom, cities.geom);`

does a spatial join on the two tables, using the `geom` (geometry) column.

Unfortunately they aren't implemented in `pandasql`, so we can't practice them here.

## A Python-based stack
We could have done all of these operations just as easily in basic `pandas`. Normally, there's no need to use `pandasql`. The point here is to demonstrate the SQL syntax, so that you can query other databases. 

A broader point is that Python is ideally suited to gluing together different operations. Technical and scientific software has Python "bindings" — you can access their functionality from within Python. For example:
* Postgres and other databases, via `psycopg2`
* Statistical software: Stata and R (use the `rpy2` library), or Stan (use `pystan`) if you are a Bayesian
* GIS software: QGIS, ArcGIS
* System-level functions like copying files and opening a web browser

## For more information
I highly recommend PostGIS in Action as an excellent introduction to PostGIS, with some basic background on PostgreSQL. [The UCLA library has a hard copy and ebook.](https://ucla.on.worldcat.org/oclc/910352485) For PostgreSQL and SQL more generally, there are many free online courses, [such as this one at UC Davis](https://www.coursera.org/specializations/learn-sql-basics-data-science).

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Databases are more complicated to work with than pandas dataframes. But they are a good solution if you have a lot of data, more than one concurrent user, or requirements for data validation and integrity.</li>
  <li>SQL uses different syntax, but the conceptual framework is similar to querying a pandas dataframe.</li>
  <li>Python can be used as a glue that holds your data collection and analysis pipeline together.</li>
</ul>
</div>